## Project Setup

In [127]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import pendulum as pen
import seaborn as sns
from pathlib import Path
from datetime import datetime as dt

In [128]:
pd.plotting.register_matplotlib_converters()

In [129]:
datasets_dir = str(Path(os.path.abspath('')).parent.joinpath("datasets"))

## COVID-19 NYT Dataset

In [130]:
covid_df = pd.read_csv(f"{datasets_dir}/covid-19_us_states.csv", parse_dates=True)

In [131]:
covid_df.head(10)

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
5,2020-01-25,California,6,1,0
6,2020-01-25,Illinois,17,1,0
7,2020-01-25,Washington,53,1,0
8,2020-01-26,Arizona,4,1,0
9,2020-01-26,California,6,2,0


In [132]:
covid_df['month_year'] = covid_df.apply(lambda row: dt.fromisoformat(row['date']).strftime('%b/%y'), axis=1)

In [135]:
covid_df.head(5)

,date,state,fips,cases,deaths,month_year
0,2020-01-21,Washington,53,1,0,Jan/20
1,2020-01-22,Washington,53,1,0,Jan/20
2,2020-01-23,Washington,53,1,0,Jan/20
3,2020-01-24,Illinois,17,1,0,Jan/20
4,2020-01-24,Washington,53,1,0,Jan/20
